In [84]:
import matplotlib.pyplot as plt
import numpy as np
import os

def get_column_from_pgm(photo_path):
    with open(photo_path, 'rb') as pgmf:
        im = plt.imread(pgmf)
    face_column = np.ndarray.flatten(im)
    return np.reshape(face_column, (-1, 1))

def get_faces_matrix(dir_with_faces):
    files = os.listdir(dir_with_faces)
    result = np.zeros(192*168)
    for filename in files:
        original_photo_path = os.path.join(dir_with_faces, filename)
        face_column = get_column_from_pgm(original_photo_path)
        result = np.column_stack((result, face_column))
    return result[:, 1:]
training_faces = get_faces_matrix('./training_faces')
print(training_faces)

[[ 80. 107.  89. ...  55.  71.  16.]
 [ 80. 101.  92. ...  60.  72.  22.]
 [ 84. 100.  93. ...  62.  72.  29.]
 ...
 [ 12.  48.  17. ...  10.  34.  13.]
 [ 12.  36.  23. ...   3.  33.   9.]
 [ 11.  24.  20. ...   6.  32.   8.]]


In [85]:
average_face = np.mean(training_faces,axis=1)
normalized_faces = training_faces - np.tile(average_face,(training_faces.shape[1],1)).T

In [86]:
def simultaneous_power_iteration(A, k):
    n, m = A.shape
    Q = np.random.rand(n, k)
    Q, _ = np.linalg.qr(Q)
    Q_prev = Q
    for i in range(1000):
        Z = A.dot(Q)
        Q, R = np.linalg.qr(Z)
        err = ((Q - Q_prev) ** 2).sum()
        Q_prev = Q
        if err < 1e-3:
            break
    return np.diag(R), Q 

In [101]:
def reduced_svd_using_qr(A):
    eigenvalues, V = simultaneous_power_iteration(A.T @ A, 800)
    sigma = np.sqrt(eigenvalues)
    idx = np.argsort(eigenvalues)[::-1]  
    sigma = sigma[idx] 
    rank_of_A = np.linalg.matrix_rank(A)
    V = V[:,idx]
    sigma = sigma[:rank_of_A]
    V = V[:, :rank_of_A]
    U = (A @ V) @ np.linalg.inv(np.diag(sigma))
    return U, sigma, V.T

In [94]:
def reduced_svd(A):
  '''
  Input: any matrix A
  Returns: tuple of matrix U, array of singular values and matrix V.T 
  '''
  # your code here
  rank_of_A = np.linalg.matrix_rank(A)

  symmetric_matrix_V = A.T @ A

  eigenvalues_V, eigenvectors_V = np.linalg.eigh(symmetric_matrix_V)
  eigenvalues_V[eigenvalues_V < 0] = 0

  idx = np.argsort(eigenvalues_V)[::-1]   
  V = eigenvectors_V[:,idx]
  V = V[:,:rank_of_A]
  
  sigma = np.sort(np.sqrt(eigenvalues_V))[::-1]
  sigma = sigma[:rank_of_A]
  U = (A @ V) @ np.linalg.inv(np.diag(sigma))
  return U, sigma, V.T


def k_rank_approximation(U, sigma, VT, k):
  return U[:,:k], sigma[:k], VT[:k,:]

In [99]:
U, sigma, VT = reduced_svd(normalized_faces)
sigma_VT = np.diag(sigma) @ VT

[[ 0.00343457  0.00304175  0.00398152 ...  0.00601772  0.00469893
  -0.00053817]
 [ 0.00329048  0.00371018  0.00411167 ...  0.00624599  0.00545016
   0.00132812]
 [ 0.00337204  0.00384763  0.00434172 ...  0.00646004  0.00551874
   0.00052141]
 ...
 [ 0.00501834 -0.01014168  0.00013672 ... -0.0112793   0.00519923
  -0.01621109]
 [ 0.00484095 -0.01052149  0.0006658  ... -0.01220265  0.00607102
  -0.01777604]
 [ 0.00442375 -0.01067183  0.00096258 ... -0.01192644  0.00453978
  -0.01721329]]
(37,)
[[ 0.05182931  0.25187842 -0.04966201 ... -0.13226594 -0.16269672
  -0.14960111]
 [ 0.10495275  0.19155023  0.17096491 ...  0.08717483  0.06530061
   0.22152499]
 [-0.03183729  0.28492724  0.1956708  ... -0.12593036  0.15419072
  -0.03947824]
 ...
 [-0.18068118 -0.05352285  0.1047802  ...  0.04223412 -0.00611732
   0.08914696]
 [-0.08510075 -0.09979134 -0.035749   ... -0.41678238  0.16426887
  -0.19922508]
 [ 0.13525504 -0.06355251  0.05162284 ...  0.61310042  0.02880938
  -0.08481034]]


In [103]:
U_qr, sigma_qr, VT_qr = reduced_svd_using_qr(normalized_faces)

[[-0.00343457 -0.00304176 -0.00349351 ... -0.00813654 -0.00296895
  -0.00053817]
 [-0.00329048 -0.00371018 -0.00364698 ... -0.00835741 -0.00373259
   0.00132812]
 [-0.00337204 -0.00384764 -0.00387902 ... -0.00875669 -0.00368064
   0.00052141]
 ...
 [-0.00501834  0.01014168 -0.00041222 ...  0.00780805 -0.00573952
  -0.01621109]
 [-0.00484095  0.01052149 -0.00097106 ...  0.00860099 -0.00675736
  -0.01777604]
 [-0.00442375  0.01067183 -0.00126172 ...  0.00908682 -0.00545058
  -0.01721329]]
[22741.95996108 10411.59604499  9583.18645704  9231.89963857
  7817.51009449  6905.75596229  6174.95935432  6081.90064304
  5411.69685931  4972.37954976  4819.06009839  4583.83591367
  4550.09325724  3988.58591418  3959.36881042  3741.45887411
  3704.05150876  3544.30304826  3490.56350413  3434.37581909
  3174.04900535  3134.29465067  3071.00155407  2995.25237742
  2829.75787357  2749.48077663  2703.11686316  2664.58960005
  2533.63742562  2469.07318822  2394.13266619  2332.18552998
  2246.70679465  210

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_19788\810654855.py:3: RuntimeWarning: invalid value encountered in sqrt
  sigma = np.sqrt(eigenvalues)


In [72]:
U_k, sigma_k, VT_k = k_rank_approximation(U, sigma, VT, 1600)

In [73]:
test_face = training_faces[:,0] - average_face
projected_face_coord = U.T @ test_face

In [75]:
for i in range(5):
    print(np.linalg.norm(projected_face_coord - sigma_VT[:, i]))

3.2098944125720547e-11
13589.708458977348
13351.333716149871
15119.967757902148
16120.32862568256
